<a href="https://colab.research.google.com/github/VOX304/SchoolChatbot/blob/main/Preprocessing/Colab_preprocessing/PdfImgTable_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python packages

In [1]:
!pip install -q ultralytics pdf2image torch torchvision
!pip install PyMuPDF pandas img2table
!pip install paddlepaddle paddlepaddle-gpu

In [2]:
!pip install paddleocr

In [3]:
!pip uninstall -y opencv-contrib-python opencv-python
!pip uninstall -y opencv-contrib-python-headless
!pip install opencv-contrib-python

Found existing installation: opencv-contrib-python 4.11.0.86
Uninstalling opencv-contrib-python-4.11.0.86:
  Successfully uninstalled opencv-contrib-python-4.11.0.86
Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Successfully uninstalled opencv-python-4.11.0.86
Found existing installation: opencv-contrib-python-headless 4.11.0.86
Uninstalling opencv-contrib-python-headless-4.11.0.86:
  Successfully uninstalled opencv-contrib-python-headless-4.11.0.86
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (69.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paddleocr 2.10.0 requires opencv-python, which is not installed.


In [4]:
!pip install opencv-contrib-python-headless

  Using cached opencv_contrib_python_headless-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_contrib_python_headless-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (56.1 MB)


In [5]:
import cv2
print("ximgproc available:", hasattr(cv2, "ximgproc"))

ximgproc available: True


In [6]:
import cv2

# Check if niBlackThreshold exists in OpenCV
if hasattr(cv2.ximgproc, "niBlackThreshold"):
    print("✅ niBlackThreshold is available!")
else:
    print("❌ niBlackThreshold is NOT available!")


✅ niBlackThreshold is available!


In [7]:
import cv2
print(cv2.__version__)

4.11.0


# Code

## Import and dowload

In [8]:
import io

In [9]:
import os
import fitz  # PyMuPDF
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from paddleocr import PaddleOCR
from img2table.document import Image as TableImage
from img2table.ocr import PaddleOCR as Img2TableOCR
from img2table.tables.objects.extraction import ExtractedTable


In [10]:
import torch
import logging

# 🔧 Patch torch.load to bypass weight loading errors
orig_torch_load = torch.load

def torch_wrapper(*args, **kwargs):
    logging.warning("[comfyui-unsafe-torch] `torch.load` is forcibly loading full weights.")
    kwargs['weights_only'] = False
    return orig_torch_load(*args, **kwargs)

torch.load = torch_wrapper

from ultralytics import YOLO

# ✅ Load YOLOv8 table detection model
model = YOLO("/content/sample_data/TableDetection.pt")

# Check if model is loaded successfully
print("✅ YOLOv8 Table Detection Model Loaded Successfully!")

ocr = PaddleOCR(use_angle_cls=True, lang="vi")
img2table_ocr = Img2TableOCR(lang="vi")


✅ YOLOv8 Table Detection Model Loaded Successfully!
[2025/03/25 08:12:28] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/latin/latin_PP-OCRv3_rec_infer', rec_image_inverse=True,

## Table Processing

In [11]:

def detect_tables(img):
    results = model(img)  # YOLOv8 table detection
    table_boxes = []

    for r in results:
        for box in r.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box)
            table_boxes.append((x1, y1, x2, y2))

    return table_boxes

def extract_table(img, table_boxes):
    extracted_tables = []

    for box in table_boxes:
        x1, y1, x2, y2 = box
        table_img = img[y1:y2, x1:x2]

        # Convert NumPy array to bytes
        _, buffer = cv2.imencode(".png", table_img)
        table = TableImage(io.BytesIO(buffer)).extract_tables(ocr=img2table_ocr, borderless_tables=True)

        if isinstance(table, list) and table:
            extracted_tables.append(table[0])  # Assuming one table per detection

    return extracted_tables

def format_table_as_markdown(extracted_table: ExtractedTable):
    if extracted_table.df is not None:
        return extracted_table.df.to_markdown(index=False)
    return "No structured table extracted."

In [12]:
def pdf_to_images(pdf_path, zoom=3.0):
    doc = fitz.open(pdf_path)
    md_output = ""

    for i, page in enumerate(doc):
        # Convert page to image (No storage, process immediately)
        pix = page.get_pixmap(matrix=fitz.Matrix(zoom, zoom))
        img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)

        # Convert grayscale to BGR if needed
        if img.shape[-1] == 1:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

        # Detect tables directly
        table_boxes = detect_tables(img)
        extracted_tables = extract_table(img, table_boxes)

        for table in extracted_tables:
            md_output += format_table_as_markdown(table) + "\n\n"

    # Save extracted tables as Markdown
    with open("pdf_table.md", "w", encoding="utf-8") as f:
        f.write(md_output)

    print("✅ Markdown file saved as output_table.md")

In [13]:
pdf_path = "/content/sample_data/HD1860 công tác xét tuyển đào tạo ĐH, CĐ năm 2024_20252211126.pdf"
pdf_to_images(pdf_path)


0: 640x480 (no detections), 1398.1ms
Speed: 7.3ms preprocess, 1398.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 918.0ms
Speed: 6.7ms preprocess, 918.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 1526.7ms
Speed: 5.8ms preprocess, 1526.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 1125.0ms
Speed: 11.5ms preprocess, 1125.0ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 895.7ms
Speed: 12.0ms preprocess, 895.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 790.2ms
Speed: 6.8ms preprocess, 790.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 478.3ms
Speed: 5.3ms preprocess, 478.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 497.6ms
Speed: 4.5